In [15]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages/') 
sys.path.append('../')
import RNA 
from utils.sequence_utils import generate_random_mutant
from utils.model_architectues import Linear, NLNN, CNNa
from models.Noisy_models.Neural_network_models import NN_model
from models.Ground_truth_oracles.RNA_landscape_models import RNA_landscape_constructor
from explorers.dqn_explorer import DQN_Explorer 
from explorers.bo_explorer import BO_Explorer
from evaluators.Evaluator import Evaluator

LANDSCAPE_TYPES ={"RNA": [2,3], 'TF':[1]}
mutation_rate = 0.5
RNA_alphabet = "UCGA" 
landscape_constructor = RNA_landscape_constructor()
#load a pre-specified landscapes  (here, index 0), you can also specify to just take a few of them
landscape_constructor.load_landscapes("../data/RNA_landscapes/RNA_landscape_config.yaml", landscapes_to_test=[0])
landscape_generator = landscape_constructor.generate_from_loaded_landscapes()
landscape_object=next(landscape_generator)
wt = landscape_object['starting_seqs']['startRNAL14_3']
landscape_oracle = landscape_object['landscape_oracle']

training_sequences = list(set([generate_random_mutant(wt, 0.2, RNA_alphabet) for i in range(0,500)]))
cnn_architecture=CNNa(len(wt), alphabet=RNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

[autoreload of explorers.bo_explorer failed: Traceback (most recent call last):
  File "/home/alexander/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/alexander/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 450, in superreload
    update_generic(old_obj, new_obj)
  File "/home/alexander/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 387, in update_generic
    update(a, b)
  File "/home/alexander/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 357, in update_class
    update_instances(old, new)
  File "/home/alexander/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 312, in update_instances
    update_instances(old, new, obj.__dict__, visited)
  File "/home/alexander/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 312, in update_instan

B1L14RNA1 loaded


In [16]:
num_of_rounds = 5

# evaluate DQN Explorer 
dqn_explorer = DQN_Explorer(seq_len=14)
dqn_explorer.debug = True
dqn_explorer.set_model(noisy_nn_model)
dqn_explorer.run(num_of_rounds, verbose=True)

round: 0, cost: 357, evals: 0, top: 0.40643708033826426
Q_Network(
  (linear1): Linear(in_features=112, out_features=56, bias=True)
  (bn1): BatchNorm1d(56, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=56, out_features=14, bias=True)
  (bn2): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=14, out_features=1, bias=True)
)
round: 1, cost: 400, evals: 43, top: 0.4091832610777826
round: 2, cost: 455, evals: 98, top: 0.6041632197437043
round: 3, cost: 470, evals: 113, top: 0.6041632197437043
round: 4, cost: 489, evals: 132, top: 0.6041632197437043


In [17]:
evaluator_dqn=Evaluator(dqn_explorer,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.adaptivity, num_starts=1)

loading landscapes RNA: [2, 3], TF:[1]
B1L100RNA1 loaded
B1L14RNA2 loaded
1 TF landscapes loaded.
loading complete
Running on B1L100RNA1
start seq startRNAL100_0


../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)


Running on B1L14RNA2
start seq startRNAL14_0


KeyboardInterrupt: 

In [5]:
# evaluate BO Explorer 
bo_explorer = BO_Explorer(seq_len=14)
bo_explorer.debug = True
bo_explorer.set_model(noisy_nn_model)
bo_explorer.run(num_of_rounds, verbose=True)

round: 0, cost: 336, evals: 0, top: 0.6151479950813181
round: 1, cost: 417, evals: 82, top: 0.6453561665444114
round: 2, cost: 476, evals: 141, top: 0.6453561665444114
round: 3, cost: 513, evals: 179, top: 0.6453561665444114
round: 4, cost: 552, evals: 218, top: 0.6453561665444114


In [ ]:
evaluator_bo=Evaluator(bo_explorer,landscape_types=LANDSCAPE_TYPES, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.adaptivity, num_starts=1)